In [26]:
import pandas as pd
import numpy as np
from processer import preprocessSmilesList, fragmentSmilesList, getSelfiesToks, augSmiles
from multiprocesspandas import applyparallel
from rdkit import Chem

In [27]:
## these smiles data are provided by grisoni github (BIMODAL work)
rawData= pd.read_csv('data/chembl_smiles.tar.xz', 
                     compression="xz", skipfooter= 1, engine='python').astype(str).squeeze()

In [28]:
rawData

0                  CSc1nn(-c2cccc(C)c2)c2cc(C3=CCNCC3)ccc21
1                  CSc1nn(-c2cccc(F)c2)c2cc(C3=CCNCC3)ccc21
2                  CSc1nn(-c2ccc(C)cc2)c2cc(C3=CCNCC3)ccc21
3         O=C(c1ccc2c3c(cnn3C3CCCC3)c(=O)[nH]c2c1)N1CCc2...
4         Cc1cc2[nH]c(=O)c3cnn(C4CCOCC4)c3c2cc1C(=O)N1CC...
                                ...                        
677039    COc1cccc(-c2ccc3c(c2)C2(COC(N)=N2)C2(COC2)C(C)...
677040    CC1(C)Oc2ccc(-c3cc(Cl)cnc3F)cc2C2(COC(N)=N2)C1...
677041       NC1=NC2(c3cc(-c4cncnc4)ccc3OCC23CC3)C(F)(F)CS1
677042    CS(=O)(=O)Nc1ccc(OC[C@@H](O)CN2CCN(c3ccc(Cl)cc...
677043    O[C@H](COc1ccc2[nH]ncc2c1)CN1CCN(c2ccc(Cl)cc2)CC1
Name: chembl_smiles.csv                                                                                   000664  001750  001750  00237257540 13547162144 014727  0                                                                                                    ustar 00robin                           robin                           00000

In [29]:
prepData = preprocessSmilesList(rawData)

100%|████████████████████████████████████████████████████████████████████████████████████| 677044/677044 [00:01<00:00, 511166.52it/s]


In [30]:
# so much data are discarded. Optical stereochemical is kept
prepData.shape

(340301,)

In [31]:
pathData = "experiments/data/"

In [32]:
grisoni_fragSmiles = fragmentSmilesList(prepData)

Process ForkPoolWorker-257:
Process ForkPoolWorker-255:
Exception ignored in: <function _releaseLock at 0x7f03c43962a0>
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 237, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 
Process ForkPoolWorker-256:
Process ForkPoolWorker-258:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tox/.local/lib/python3.11/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/tox/.local/lib/python3.11/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tox/.local/lib/python3.11/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tox/.local/lib/python3.11/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)


KeyboardInterrupt: 

In [ ]:
grisoni_fragSmiles_10_32 = grisoni_fragSmiles.query('nToks >=10 and nToks<=32')

In [ ]:
grisoni_train = grisoni_fragSmiles_10_32.copy()
grisoni_train.rename(columns={'tokens':'fragsmiles'},inplace=True)
grisoni_train.drop(columns='nToks', inplace=True)
grisoni_train['selfies']=grisoni_train['smiles'].apply_parallel(getSelfiesToks, num_processes=30)

100%|████████████████████████████████████████████████████████████████████████████████████| 270408/270408 [00:00<00:00, 412406.58it/s]


In [ ]:
grisoni_train

,smiles,fragsmiles,selfies
0,CSc1nn(-c2cccc(C)c2)c2cc(C3=CCNCC3)ccc12,"(C, S, <6>, c1ccc2[nH]ncc2c1, <4>, <1>, (, <0>...","([C], [S], [C], [=N], [N], [Branch1], [N], [C]..."
1,CSc1nn(-c2cccc(F)c2)c2cc(C3=CCNCC3)ccc12,"(C, S, <6>, c1ccc2[nH]ncc2c1, <4>, <1>, (, <0>...","([C], [S], [C], [=N], [N], [Branch1], [N], [C]..."
2,CSc1nn(-c2ccc(C)cc2)c2cc(C3=CCNCC3)ccc12,"(C, S, <6>, c1ccc2[nH]ncc2c1, <4>, <1>, (, <0>...","([C], [S], [C], [=N], [N], [Branch1], [N], [C]..."
3,O=C(c1ccc2c(c1)[nH]c(=O)c1cnn(C3CCCC3)c12)N1CC...,"(Cl, <0>, c1ccc2c(c1)CCN2, <8>, C=O, <5>, O=c1...","([O], [=C], [Branch2], [Ring2], [C], [C], [=C]..."
4,Cc1cc2[nH]c(=O)c3cnn(C4CCOCC4)c3c2cc1C(=O)N1CC...,"(C, <5>, O=c1[nH]c2ccccc2c2[nH]ncc12, <6>, <10...","([C], [C], [=C], [C], [NH1], [C], [=Branch1], ..."
...,...,...,...
677034,OC[C@@H]1CCCN1Cc1nc2ccccc2n1Cc1ccc(Cl)cc1,"(O, C, <4S>, C1CCNC1, <3>, C, <5>, c1ccc2[nH]c...","([O], [C], [C@@H1], [C], [C], [C], [N], [Ring1..."
677035,Cc1ccc2nc(CN3CCCC3)n(Cc3ccc(Cl)cc3)c2c1,"(C1CCNC1, <3>, C, <5>, c1ccc2[nH]cnc2c1, <4>, ...","([C], [C], [=C], [C], [=C], [N], [=C], [Branch..."
677041,NC1=NC2(c3cc(-c4cncnc4)ccc3OCC23CC3)C(F)(F)CS1,"(F, <3>, C1=NC2(CCS1)c1ccccc1OCC21CC1, <0>, <3...","([N], [C], [=N], [C], [Branch2], [Ring1], [N],..."
677042,CS(=O)(=O)Nc1ccc(OC[C@@H](O)CN2CCN(c3ccc(Cl)cc...,"(Cl, <3>, c1ccccc1, <0>, <5>, C1CNCCN1, <2>, C...","([C], [S], [=Branch1], [C], [=O], [=Branch1], ..."


In [ ]:
grisoni_fragSmilesAug5 = fragmentSmilesList( grisoni_train["smiles"], augmentation=5 )

100%|███████████████████████████████████████████████████████████████████████████████████| 1352040/1352040 [00:26<00:00, 50422.03it/s]


In [ ]:
grisoni_SmilesAug5 = grisoni_train["smiles"].apply_parallel(augSmiles)
grisoni_SmilesAug5 = grisoni_SmilesAug5.explode()

100%|████████████████████████████████████████████████████████████████████████████████████| 270408/270408 [00:00<00:00, 487577.85it/s]


In [ ]:
grisoni_trainAug5 = grisoni_SmilesAug5.to_frame(name='smiles')

In [ ]:
grisoni_trainAug5['fragsmiles']=grisoni_fragSmilesAug5['tokens']

In [38]:
grisoni_trainAug5['selfies']=grisoni_trainAug5['smiles'].apply_parallel(getSelfiesToks, num_processes=30)

Exception ignored in: <built-in method acquire of _thread.lock object at 0x7f03c272fbc0>
Traceback (most recent call last):
  File "/home/tox/.local/lib/python3.11/site-packages/multiprocess/popen_fork.py", line 66, in _launch
    self.pid = os.fork()
               ^^^^^^^^^
KeyboardInterrupt: 
100%|██████████████████████████████████████████████████████████████████████████████████| 1352040/1352040 [00:02<00:00, 481103.68it/s]


In [40]:
## some molecules cant be represented more than 1/2/3 fragSmiles representation
grisoni_trainAug5[grisoni_trainAug5.isnull().any(axis=1)]

,smiles,fragsmiles,selfies
3,n1cc2c([nH]c3cc(C(=O)N4c5c(cc(cc5)Cl)CC4)ccc3c...,NaN,"([N], [=C], [C], [C], [Branch2], [Branch1], [R..."
3,C(N1c2ccc(Cl)cc2CC1)(=O)c1ccc2c3n(ncc3c(=O)[nH...,NaN,"([C], [Branch1], [P], [N], [C], [=C], [C], [=C..."
3,c1cc2c3c(cnn3C3CCCC3)c([nH]c2cc1C(=O)N1c2c(CC1...,NaN,"([C], [=C], [C], [C], [=C], [Branch1], [=N], [..."
6,C1(CCOCC1)n1ncc2c([nH]c3c(ccc(c3)C(=O)N3CCc4c3...,NaN,"([C], [Branch1], [Branch2], [C], [C], [O], [C]..."
6,c1nn(C2CCOCC2)c2c3ccc(C(N4CCc5cc(Cl)ccc45)=O)c...,NaN,"([C], [=N], [N], [Branch1], [=Branch2], [C], [..."
...,...,...,...
677033,c12nc(n(Cc3ccc(cc3)Cl)c2cccc1)CN1CCCC1,NaN,"([C], [N], [=C], [Branch2], [Ring1], [=Branch2..."
677033,c12nc(n(c1cccc2)Cc1ccc(cc1)Cl)CN1CCCC1,NaN,"([C], [N], [=C], [Branch2], [Ring1], [=Branch2..."
677034,c1(n(c2c(cccc2)n1)Cc1ccc(Cl)cc1)CN1[C@@H](CCC1)CO,NaN,"([C], [Branch2], [Ring1], [=N], [N], [Branch1]..."
677034,[C@@H]1(N(Cc2n(Cc3ccc(cc3)Cl)c3c(n2)cccc3)CCC1)CO,NaN,"([C@@H1], [Branch2], [Ring2], [=Branch2], [N],..."


In [ ]:
from sklearn.model_selection import KFold

In [41]:
def saveSplittedDataset(dataset, suffix=''):

    indexes = dataset.index.unique()

    kf=KFold(n_splits=5, shuffle=True, random_state=0)

    kf = kf.split( indexes )

    for fold,(trainIdx,testIdx) in enumerate(kf):
        train = dataset.loc[indexes[trainIdx]]
        test = dataset.loc[indexes[testIdx]]

        train.to_csv(pathData+f"grisoni_train{suffix}_{fold}.tar.xz", compression="xz", index=False)
        test.to_csv(pathData+f"grisoni_test{suffix}_{fold}.tar.xz", compression="xz", index=False)

In [ ]:
saveSplittedDataset(grisoni_train)

In [ ]:
saveSplittedDataset(grisoni_trainAug5, suffix='Aug5')